# Observation Targets Assessor
This notebook aims to ingest a table of source/observation specs with their achievability rating per detector model (as generated by prior notebook), and convolve with the Science STM objectives, each of which involves one or more of those source/observation specs. The output should be a similar table to the input, but with the source/observvation specs replaced by objectives.

To run this on Google Colab:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnGBaker/GWI-metrics/blob/main/Notebooks/STM_Science_Assessor.ipynb)

### Some basic setup

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/JohnGBaker/GWI-metrics.git
    src='GWI-metrics/src/'
else:
    src='../src/'
!mkdir -p '../plots/'

# Import
import os
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(src)
import constants
import metrics
import sources
import concepts
import subsystems
import background
import pandas as pd
from glob import glob
import re
from astropy.coordinates import Angle
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u

LARGENUM = 1.0e10 # This is a hard-coded number to beat

### Define the set of concepts
Here we load in a set of the pre-defined concepts, or you can define your own

In [2]:
missionNames = (
    'LISACBE',
    'LISASciRDv1',
    'TwinLISA',
    'LISAGrande',
    'LISAU',
    'GoBIGLISA',
    'ALIA',
    'ALIAlowL')
missions=[concepts.menu[mission] for mission in missionNames]

Nmissions = len(missionNames)

for mission in missions:
    mission=background.add2model(mission)
#model = concepts.LISACBE.copy()
#modelName = model.get('label')
#model = background.add2model(model)     # add galactic background model

### Load the observation target data
Load information about the target sources and the observation quality targets from the STM.
The data are from https://nasa.sharepoint.com/:x:/t/GravitationalWaveImager/EaaeMC7L-2NJpFCQloYbSnoBDZag_cvFWR5_BjRoFAD6Tw?e=uWr4mU

In [3]:
# Read in the most recent spec file produced by other Notebook:
datadir='../Data/'
files=glob(datadir+'ScoredObservations_res_*.csv')
files.sort(key=os.path.getmtime)
specfile=files[-1]
print('Most recent specs file is:',specfile)

# Are objective names actually stored?
# A: not originally -- had to re-run earlier notbook with the "index=None" removed from the CVS save command

speclist = []
with open(specfile) as f:
    for row in f:
        speclist.append(row.split(",")[0])

speclist.pop(0) # remove the top item, which is empty "corner" element of table
print("specs are: ",speclist)

Nspecs = len(speclist)

#specarray = np.loadtxt(specfile,delimiter=',',skiprows=1,usecols = range(1,Nmissions+1))

rawspecarray = np.genfromtxt(specfile,delimiter=',')

specarray=rawspecarray[1:Nspecs+1,1:Nmissions+1]
#print("first row of specarray is ",specarray[0,])



    

Most recent specs file is: ../Data/ScoredObservations_res_2022_06_21.csv
specs are:  ['1.1.b', '1.1.c', '1.1.d', '1.1.e', '1.2.b', '1.3.b', '1.3.c', '1.3.d', '1.3.e', '2.1.a', '2.1.b', '2.1.c', '2.1.d', '2.1.e', '2.2.b', '2.3.b', '3.1.a', '3.2.a', '3.2.b', '3.3.a', '3.3.b', '3.3.c', '3.3.d', '3.3.e', '3.4.a', '4.1.a', '4.1.b', '4.1.c', '4.1.d', '5.1.a', '5.1.b', '5.1.c', '5.1.d', '5.2.a', '5.2.b', '5.3.a', '5.3.b', '5.4.a', '5.4.b', '5.5.a', '5.5.b', '5.5.c', '5.6.a']


In [7]:
file=glob('../Data/ScienceSTM*.csv')[0]
print('Science STM file is:',file)
#Read file
df=pd.read_csv(file,header=1)
print(len(df),'rows read from file.')

if True:
    #Drop empty rows
    for i,row in df.iterrows():
        if row.isnull().all():
            df=df.drop(i)
    #Fill empties in first column with values above

    nrows = len(df. index) 

    newdata = np.zeros((nrows,Nmissions))

    val=float('nan')
    
    col='Astrophysical Parameters (Level 1 Measurement Req)'
    requirementNames=df[col].values
    print("requirement Names are ",requirementNames)

    analysisVal=float('nan')
    analysisCol='Specific Analyses'
    analysisNames=df[analysisCol].values

    print("analyses are: ",analysisNames)
    
    for i,row in df.iterrows(): #each row should be a specific analysis in analysisNames
        if row.isnull()[col]:
            df.loc[i,col]=val
        else:
            val=df.loc[i,col]

        if row.isnull()[analysisCol]:
            df.loc[i,analysisCol]=analysisVal
        else:
            analysisVal=df.loc[i,analysisCol]    
            
        #print("this row has astrophysical parameter ",val)

        bestspecres_here = LARGENUM*np.ones(Nmissions)

        # parse to get values in form of 2.3.b == <single digit> + dot + <single digit> + dot + <single letter>
        # or perhaps just a comma-separated list that can be compared with other file
        speccases_here = val.split(",")

        Nspeccases_here = len(speccases_here)
        for ispeccases_here in range(Nspeccases_here):
            speccase_here = (speccases_here[ispeccases_here]).strip() # remove leading whitespace
            # look for match with list of specs read in earlier
            # if it's present, combine with existing data for this requirement -- how?
            # for now: try smallest number
            specres_here = 2.0*LARGENUM*np.ones(Nmissions)
            try:
                specidx = speclist.index(speccase_here)
                specres_here = specarray[specidx,:]
            except ValueError:
                print("specification ",speccase_here," not generated by earlier Notebook! Ignoring")
           
            bestspecres_here = np.minimum(bestspecres_here,specres_here)

        #print("For this requirement, the best spec results for each detector are ",bestspecres_here)
        newdata[i,:] = bestspecres_here
        # This is a Ndetectors-length array for this particular Specific Analysis


Science STM file is: ../Data/ScienceSTM-21-Jun-23.csv
77 rows read from file.
requirement Names are  ['2.1.b' '2.3.b' '1.1.f' '1.2.c' nan nan nan nan '2.1.f, 2.3.c' '1.3.f'
 '1.1.g' '1.2.d' '4.1.e' '3.1.b, 3.2.b, 3.3.f, 3.4.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b' nan nan nan nan nan nan
 nan nan nan '2.1.c, 2.1.d, 2.1.e' '1.3.c, 1.3.d, 1.3.e'
 '1.1.c, 1.1.d, 1.1.e' nan '4.1.c, 4.1.d' nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan 'SNR O(1000)' '5.5.c' nan nan nan
 '0.11 Hz -> 0.22 Hz, (find merger-time SNR requirement); 1.3.a (?)'
 '(choose separation; add frequencies), with SNR O(1000)'
 '0.5 Hz -> 1.0 Hz (find merger-time SNR requirement)'
 '(choose separation; add frequencies), with SNR O(1000)'
 '0.29 Hz -> 0.58 Hz,  (find merger-time SNR requirement)'
 '(choose separation; add frequencies), with SNR O(1000)' nan nan nan
 '2.1.a, 2.1.b, 2.1.c, 2.3.a, 2.3.b' '2.1.a, 2.1.b, 2.1.c'
 '2.1.a, 2.1.b, 2.1.d, 2.3.a'
 '3.3.c (less pessimistic model), 3.3.

### Perform the observations and assessment
We first compute the SNR and then the angular resolution

In [5]:
adfnew = pd.DataFrame(newdata,index=requirementNames,columns=missionNames)

#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]


s=adfnew.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Net performance")
display(s)

if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredRequirements_res_'+dates+'.csv'
    adfnew.to_csv(outfile,index=False)
    print('Wrote to file:',outfile)


requirement Names are  ['2.1.b' '2.3.b' '1.1.f' '1.2.c' '1.2.c' '1.2.c' '1.2.c' '1.2.c'
 '2.1.f, 2.3.c' '1.3.f' '1.1.g' '1.2.d' '4.1.e'
 '3.1.b, 3.2.b, 3.3.f, 3.4.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b'
 '5.1.e, 5.2.d, 5.3.d, 5.4.d, 5.5.c, 5.6.b, 5.7.b' '2.1.c, 2.1.d, 2.1.e'
 '1.3.c, 1.3.d, 1.3.e' '1.1.c, 1.1.d, 1.1.e' '1.1.c, 1.1.d, 1.1.e'
 '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d'
 '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d'
 '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d'
 '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c, 4.1.d' '4.1.c

KeyError: '`Styler.apply` and `.applymap` are not compatible with non-unique index or columns.'

Wrote to file: ../Data/ScoredRequirements_res_2022_06_23.csv


### Show assessments

### Make some plots

#### Plot sensitivity curves and waveform
Here we scale the waveform by delta-f, which is probably not quite right